# Pandas e Plotly

Exemplos de análise de dados dos CSV gerados pelo programa `Heurística_HDB`.

Utilizaremos os recursos avançados de análise de dataframes com pandas e em seguida vamos utilizar a biblioteca Plotly para gerar gráficos interativos.

---


## Importando as bibliotecas e módulos

In [16]:
# importando pandas e plotly
import pandas as pd
import plotly.express as px

## Inserindo caminho para o CSV e termo para salvar o arquivo

In [17]:
# caminho do arquivo csv
csv_path = input("Enter CSV path: ")
# nome do arquivo de saída
output_file = input("Enter output file name: ")
# colocar o nome do arquivo em minúsculo
output_file = output_file.lower()

## Criando o dataframe

A partir do caminho do CSV, vamos utilizar o método `pd.read_csv()` para gerar o dataframe.

In [18]:
# Criar um dataframe com o arquivo csv
df = pd.read_csv(f'{csv_path}', index_col=0, encoding='utf-8')
# Mostrar as 5 primeiras linhas do dataframe
df.head()

,Ano,Evento,Cidade,ST,Coordenadores,Autor(es)/Instituições,Título,Resumo
0,2013,XXVII,Natal,"001. ""De que África estamos falando (I) : per...",ALEXANDRE VIEIRA RIBEIRO (Doutor(a) - Universi...,Letícia Cristina Fonseca Destro,"Cristãos, mouros e gentios: os africanos subsa...",Quando as caravelas portuguesas se encontraram...
1,2013,XXVII,Natal,"001. ""De que África estamos falando (I) : per...",ALEXANDRE VIEIRA RIBEIRO (Doutor(a) - Universi...,Ariane Carvalho da Cruz,A africanização da guerra em Angola na segunda...,O presente trabalho analisa as formas de organ...
2,2013,XXVII,Natal,"001. ""De que África estamos falando (I) : per...",ALEXANDRE VIEIRA RIBEIRO (Doutor(a) - Universi...,Érika Melek Delgado,A Expedição Vitoriana para a África Ocidental:...,Este artigo tem como objetivo dar espaço para ...
3,2013,XXVII,Natal,"001. ""De que África estamos falando (I) : per...",ALEXANDRE VIEIRA RIBEIRO (Doutor(a) - Universi...,VANICLEIA SILVA SANTOS,O colonialismo inquisitorial: O Santo Ofício p...,O objetivo dessa apresentação é mostrar o func...
4,2013,XXVII,Natal,"001. ""De que África estamos falando (I) : per...",ALEXANDRE VIEIRA RIBEIRO (Doutor(a) - Universi...,Cristiana Ferreira Lyrio Ximenes,ANGOLA E BAHIA NAS REDES E ROTAS TRANSOCEÂNICA...,O texto apresenta algumas conexões constituída...


In [19]:
# ver as colunas do dataframe
df.columns


Index(['Ano', 'Evento', 'Cidade', 'ST', 'Coordenadores',
       'Autor(es)/Instituições', 'Título', 'Resumo'],
      dtype='object')

In [20]:
# ver os tipos de dados do dataframe
df.dtypes

Ano                        int64
Evento                    object
Cidade                    object
ST                        object
Coordenadores             object
Autor(es)/Instituições    object
Título                    object
Resumo                    object
dtype: object

## Analisar a coluna Eventos

In [21]:
df['Evento'].unique()

array(['XXVII', 'XXVIII', 'XXIX', 'XXX', 'XXXI', 'XXXII'], dtype=object)

In [22]:
# Agrupar por evento e contar
df_evento = df.groupby('Evento').size()
df_evento

Evento
XXIX      2802
XXVII     2678
XXVIII    2770
XXX       3240
XXXI      2729
XXXII     2372
dtype: int64

In [23]:
df_evento = df.groupby('Evento', sort=False).size()
df_evento

Evento
XXVII     2678
XXVIII    2770
XXIX      2802
XXX       3240
XXXI      2729
XXXII     2372
dtype: int64

In [24]:
type(df_evento)

pandas.core.series.Series

>OBS: Importante reparar que nesse caso NÃO criamos um novo dataframe com os dados agrupados e contados (size) pela coluna 'ST', mas sim um objeto Series.

In [25]:
# Podemos usar o método `reset_index()` para transformar o resultado da agregação em um dataframe
df_evento = df_evento.reset_index(name='Total de resumos')
df_evento

,Evento,Total de resumos
0,XXVII,2678
1,XXVIII,2770
2,XXIX,2802
3,XXX,3240
4,XXXI,2729
5,XXXII,2372


#### Plotly

O Plotly é uma biblioteca de visualização de dados interativa. Para utilizá-la, é necessário instalar o pacote `plotly`.

```bash
pip install plotly
```

E importar o módulo `plotly.express` como `px`.

```python
import plotly.express as px
```

Vamos criar um gráfico de barras com o Plotly. Para isso vamos utilizar o método `px.bar()`.

```python
fig = px.bar(dataframe, x='Coluna 1', y='Coluna 2')
```

In [26]:
# gráfico de barras com coluna x = Evento e y = Quantidade
fig = px.bar(df_evento, x='Evento', y='Total de resumos') 

fig.show()

In [27]:
# atualizar o layout do gráfico, incluindo título
fig.update_layout(title_text='Quantidade de resumos por ano')
fig.show()

In [28]:
#salvar o gráfico em um arquivo html e png
fig.write_html(f'{output_file}.html')
fig.write_image(f'{output_file}.png')

### Filtrar usando múltiplas colunas

In [29]:
df.columns

Index(['Ano', 'Evento', 'Cidade', 'ST', 'Coordenadores',
       'Autor(es)/Instituições', 'Título', 'Resumo'],
      dtype='object')

In [30]:
df_ano_evento = df.filter(items=['Ano', 'Evento', 'Cidade'])\
    .groupby(['Ano', 'Evento', 'Cidade'])\
    .size()
df_ano_evento

Ano   Evento  Cidade        
2013  XXVII   Natal             2678
2015  XXVIII  Florianópolis     2770
2017  XXIX    Brasília          2802
2019  XXX     Recife            3240
2021  XXXI    Rio de Janeiro    2729
2023  XXXII   São Luís          2372
dtype: int64

In [31]:
df_ano_evento = df_ano_evento.reset_index()
df_ano_evento.columns = ['Ano', 'Evento', 'Cidade', 'Total de resumos']
df_ano_evento

,Ano,Evento,Cidade,Total de resumos
0,2013,XXVII,Natal,2678
1,2015,XXVIII,Florianópolis,2770
2,2017,XXIX,Brasília,2802
3,2019,XXX,Recife,3240
4,2021,XXXI,Rio de Janeiro,2729
5,2023,XXXII,São Luís,2372


In [32]:
# gráfico de barras com coluna x = Ano e y = Quantidade
fig2 = px.bar(df_ano_evento, x='Ano', y='Total de resumos', title='Quantidade de Eventos por ano')

fig2.show()

In [33]:
# atualizar traces do eixo x: anos fixos
fig2.update_xaxes(tickvals=[2013, 2015, 2017, 2019, 2021, 2023])
fig2.show()

## gráfico de Pizza


In [34]:
fig3 = px.pie(df_ano_evento, values='Total de resumos', names='Ano', title='Eventos por ano')
fig3.show()

## Analisar a coluna Eventos e ST

In [35]:
df['ST'].unique()


array(['001. "De que África estamos falando\x94 (I) : perspectivas da pesquisa histórica e do ensino de História da África (do século XI à primeira metade do século XIX)',
       '002. "De que África estamos falando\x94 (II): perspectivas da pesquisa histórica e do ensino de História da África (do século XIX à configuração dos Estados independentes).\x94',
       '003. A dinâmica imperial e a comunicação política no Antigo Regime do mundo português, séculos XVI-XIX',
       '004. A EDUCAÇÃO E A FORMAÇÃO DA SOCIEDADE BRASILEIRA',
       '005. A empresa jesuítica: da antiga Companhia à Companhia restaurada',
       '006. A História Antiga e as fronteiras do conhecimento histórico',
       '008. A História Pública e os Públicos da História',
       '009. A instituição militar no Brasil: diálogos entre história e ciências sociais',
       '011. A Presença Indígena na História do Brasil',
       '013. Antiguidade e Modernidade: usos do passado / 018. As Formas e os Limites do Político nos E

In [36]:
# limpar valores em coluna 'ST': excluir '\r\n\t\t'
df['ST'] = df['ST'].str.replace('\r\n\t\t', '')
df['ST'].unique()

array(['001. "De que África estamos falando\x94 (I) : perspectivas da pesquisa histórica e do ensino de História da África (do século XI à primeira metade do século XIX)',
       '002. "De que África estamos falando\x94 (II): perspectivas da pesquisa histórica e do ensino de História da África (do século XIX à configuração dos Estados independentes).\x94',
       '003. A dinâmica imperial e a comunicação política no Antigo Regime do mundo português, séculos XVI-XIX',
       '004. A EDUCAÇÃO E A FORMAÇÃO DA SOCIEDADE BRASILEIRA',
       '005. A empresa jesuítica: da antiga Companhia à Companhia restaurada',
       '006. A História Antiga e as fronteiras do conhecimento histórico',
       '008. A História Pública e os Públicos da História',
       '009. A instituição militar no Brasil: diálogos entre história e ciências sociais',
       '011. A Presença Indígena na História do Brasil',
       '013. Antiguidade e Modernidade: usos do passado / 018. As Formas e os Limites do Político nos E

In [37]:
# remover '\t' do final da string na coluna 'ST'
df['ST'] = df['ST'].str.rstrip('\t')
df['ST'].unique()

array(['001. "De que África estamos falando\x94 (I) : perspectivas da pesquisa histórica e do ensino de História da África (do século XI à primeira metade do século XIX)',
       '002. "De que África estamos falando\x94 (II): perspectivas da pesquisa histórica e do ensino de História da África (do século XIX à configuração dos Estados independentes).\x94',
       '003. A dinâmica imperial e a comunicação política no Antigo Regime do mundo português, séculos XVI-XIX',
       '004. A EDUCAÇÃO E A FORMAÇÃO DA SOCIEDADE BRASILEIRA',
       '005. A empresa jesuítica: da antiga Companhia à Companhia restaurada',
       '006. A História Antiga e as fronteiras do conhecimento histórico',
       '008. A História Pública e os Públicos da História',
       '009. A instituição militar no Brasil: diálogos entre história e ciências sociais',
       '011. A Presença Indígena na História do Brasil',
       '013. Antiguidade e Modernidade: usos do passado / 018. As Formas e os Limites do Político nos E

## Filtrar e analisar por colunas específicas

In [38]:
# agrupar por 'ST' e contar ocorrências e ordenar de forma decrescente
df_st = df.groupby('ST').size().reset_index(name='Total').sort_values(by='Total',ascending=False)
df_st

,ST,Total
49,"023. Catolicismo, repressão e diversidade cult...",69
145,060/061/088: História da Loucura e da Psiquiat...,66
418,ST 034. Dimensões do Regime Vargas e seus desd...,49
296,"120/134: Produção e circulação de saberes, ide...",48
125,"054. Habitação e direito à cidade: favelas, mo...",46
...,...,...
625,ST 131. Povos indígenas e ensino de história,8
571,ST 103. Macrotempos e macroespaços: perspectiv...,8
650,ST 148. Visões da Ásia: olhares de pesquisador...,7
599,ST 115. Negacionismos e usos da história,7


In [39]:
df_ano_st = df.filter(items=['Ano', 'ST'])\
    .groupby(['Ano', 'ST'])\
    .size()\
    .reset_index(name='Total de resumos')\
    .sort_values(by='Total de resumos', ascending=False, ignore_index=True)
df_ano_st.head()

,Ano,ST,Total de resumos
0,2013,"023. Catolicismo, repressão e diversidade cult...",69
1,2013,060/061/088: História da Loucura e da Psiquiat...,66
2,2013,"120/134: Produção e circulação de saberes, ide...",48
3,2013,"075/106: História, Direito e Memória",46
4,2013,"126/127: Patrimônio Histórico, História da Art...",46


#### df_ano_st

Esse dataframe contém a quantidade total de resumos por ST em cada ano.

Vamos utilizar esses dados para gerar visualizações com Plotly.

In [40]:
# avaliar os 20 maiores valores de 'ST' e mostrar em um gráfico de barras
fig4 = px.bar(df_ano_st.head(20), x='ST', y='Total de resumos', title='Quantidade de resumos por ST', color='Ano')

fig4.show()

In [41]:
for index, row in df_ano_st.iterrows():
    if len(row['ST']) > 40:
        df_ano_st.at[index, 'ST'] = row['ST'][:40] + '...'

In [42]:
fig4 = px.bar(df_ano_st.head(15), x='ST', y='Total de resumos', title='Quantidade de resumos por ST', color='Ano')

fig4.show()

---

[← Anterior](dia3.ipynb)

[↑ Início](./README.md)